In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Input,Dense
from keras.layers import Conv2D,MaxPooling2D,Lambda,Dropout
from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.models import load_model

In [ ]:
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from cs231n.data_utils import load_CIFAR10

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=10000):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    y_train = to_categorical(y_train, num_classes=10)
    y_test = to_categorical(y_test, num_classes=10)
    # Subsample the data
#     mask = range(num_training, num_training + num_validation)
#     X_val = X_train[mask]
#     y_val = y_train[mask]
#     mask = range(num_training)
#     X_train = X_train[mask]
#     y_train = y_train[mask]
#     mask = range(num_test)
#     X_test = X_test[mask]
#     y_test = y_test[mask]

#     # Normalize the data: subtract the mean image
#     mean_image = np.mean(X_train, axis=0)
#     X_train -= mean_image
#     X_val -= mean_image
#     X_test -= mean_image

#     return X_train, y_train, X_val, y_val, X_test, y_test
    return X_train, y_train, X_test, y_test


# Invoke the above function to get our data.
# X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()
X_train, y_train, X_test, y_test = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
# print('Validation data shape: ', X_val.shape)
# print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)


In [ ]:
model = Sequential()

#model.add(BatchNormalization(mode=1, input_shape=(32,32,3)))
model.add(Lambda(lambda x: x/255.0,input_shape = X_train.shape[1:]))
model.add(Conv2D(32,(3,3),padding='same',))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))

model.add(Conv2D(64,(3,3),padding='same',))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(Dropout(0.1))
 
model.add(Conv2D(64,(3,3),padding='same',))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))
          
model.add(Conv2D(128,(3,3),padding='valid',))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))
          
model.add(Conv2D(128,(3,3),padding='valid',))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
          

model.add(Flatten())
model.add(Dense(100, activation='relu'))

# model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='softmax'))

          
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])


In [ ]:
num_train = X_train.shape[0]
subsample_index = np.random.choice(num_train, 1000)
X_subtrain = X_train[subsample_index]
y_subtrain = y_train[subsample_index]


In [ ]:
history = model.fit(x=X_subtrain, y=y_subtrain, batch_size=32, epochs=10)

In [ ]:
history = model.fit(x=X_train, y=y_train, batch_size=64, epochs=3, validation_split=0.05)

In [ ]:
evaluation = model.evaluate(x=X_test, y=y_test, batch_size=32)

## GridSearch

In [ ]:

def model(ac='relu', opz = 'adam', pd1='same', pd2='valid',dp=0, MP=True, loss='categorical_crossentropy'):
	image_input = Input(shape = (32,32,3))
	x = Lambda(lambda x: x/255.0,input_shape = X_train.shape[1:])(image_input)

	x = Conv2D(32,(3,3),padding=pd1)(x)
	x = BatchNormalization(axis=3)(x)
	x = Activation(ac)(x)

	x = Conv2D(64,(3,3),padding=pd1)(x)
	x = BatchNormalization(axis=3)(x)
	x = Activation(ac)(x)
	x = Dropout(dp)(x)

	x = Conv2D(64,(3,3),padding=pd2)(x)
	x = BatchNormalization(axis=3)(x)
	x = Activation(ac)(x)
	if MP:
		x = MaxPooling2D((2,2))(x)
	x = Dropout(dp)(x)

	x = Conv2D(128,(3,3),padding=pd2)(x)
	x = BatchNormalization(axis=3)(x)
	x = Activation(ac)(x)
	if MP:
		x = MaxPooling2D((2,2))(x)
	x = Dropout(dp)(x)

	x = Conv2D(128,(3,3),padding=pd2)(x)
	x = BatchNormalization(axis=3)(x)
	x = Activation(ac)(x)
	x = Dropout(dp)(x)

	x = Flatten()(x)
	x = Dense(100)(x)
	x = BatchNormalization(axis=1)(x)
	x = Activation(ac)(x)

	x = Dense(50)(x)
	x = BatchNormalization(axis=1)(x)
	x = Activation(ac)(x)

	out = Dense(10, activation='softmax')(x)

	model = Model(inputs=image_input, outputs=out)
	model.compile(optimizer=opz, loss =loss) 

	return model

In [ ]:
from keras.callbacks import EarlyStopping,CSVLogger,ModelCheckpoint
import itertools

optimizers = ['adam','rmsprop']
activations = ['relu','LeakyReLU']
dropouts = [0,0.1,0.2,0.3]
padding_shallow = ['same','valid']
padding_deep = ['same','valid']
# cost_fun = [loss_max,'mae',loss_mse]

# models = [mul_2014]
parameters = itertools.product(activations,optimizers,padding_shallow,padding_deep,dropouts)#,cost_fun)


    
filepath="{epoch:03d}:{val_loss:.3f}.h5"
callbacks = [EarlyStopping(monitor='val_loss',patience=6),ModelCheckpoint(filepath, monitor='val_loss',save_best_only=True)]     

with tf.device('/gpu:0'):
    for ac, opz, pd1, pd2, dp in parameters:
        model_1= model(ac, opz, pd1, pd2, dp, True, 'categorical_crossentropy')
        print('*'*100)
        print(str((ac, opz, pd1, pd2, dp))+'\n')
        model_1.fit(X_train, y_train,epochs=50,batch_size=128,validation_split=0.05 ,callbacks=callbacks,shuffle="batch")




## Test Result

In [ ]:
model = load_model('/Users/zouxiaxia/Desktop/014_0.530.h5')

In [ ]:
model.evaluate(X_test, y_test)

In [8]:
from scipy import stats

u = []
model_paths = ['2017_10_06_am:014:0.529.h5','2017_10_06_am:016:0.540.h5','2017_10_06_am:009:0.537.h5']
for p in model_paths: 
    model = load_model('/Users/zouxiaxia/Desktop/'+p)
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    u.append(y_pred)
y_pred,_ =  stats.mode(np.array(u))

test_acc = np.sum(np.equal(y_pred,np.argmax(y_test, axis=1)))/10000.0
print(test_acc) 

KeyboardInterrupt: 

In [ ]:
model.summary()

In [ ]:
for layer in model.weighted_metrics:
    print(layer)